In [1]:
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
from openai import OpenAI

In [5]:
class ChatBot:
    def __init__(self, protocol="You are a helpful assistant."):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.protocol = protocol
        self.history = [{"role": "system", "content": self.protocol}]

    def generate_response(self, prompt: str) -> str:
        self.history.append({"role": "user", "content": prompt})

        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo", # NOTE: feel free to change it to gpt-4, or gpt-4o
            messages=self.history
        )

        response = completion.choices[0].message.content
        self.history.append({"role": "assistant", "content": response})

        return response

    def get_history(self) -> list:
        return self.history

In [30]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification
from transformers import pipeline
import torch

In [32]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")


def political_bias_classifier(text: str, tokenizer=tokenizer, model=model) -> list[float]:
    inputs = tokenizer(text, return_tensors="pt")
    labels = torch.tensor([0])
    outputs = model(**inputs, labels=labels)
    loss, logits = outputs[:2]

    return logits.softmax(dim=-1)[0].tolist()

In [53]:
judge = ChatBot(protocol="You explain why the text is politically biased and provide feedback how to make it neutral.")
writer = ChatBot(protocol="Rewrite input text to be politically neutral. A feedback will be provided. Learn from the feedback. Do not show political bias. If the text is left leaning, write it to be more right leaning. And vice versa.")

In [54]:
prompt = """
What other options are there for raising $350 billion with such a negligible impact on middle-class Americans and Main Street businesses?
"""
round = 0
rounds = []
left_scores = []
left, middle, right = political_bias_classifier(prompt)
print("Pre-score:", left, middle, right)

while True:

    if left > 0.3:
        feedback = judge.generate_response(prompt)
        print("-------------------------------")
        print("Round ", round+1)
        print("Feedback: ", feedback)

        prompt = writer.generate_response(
            f"Use the feedback: {feedback}"
            f"And rewrite the prompt: {prompt}")
        left, middle, right = political_bias_classifier(prompt)
        print("Post-score:", left, middle, right)
        print("-------------------------------")

        round += 1

        rounds.append(round)
        left_scores.append(left)

    else:
        break

Pre-score: 0.46122321486473083 0.2824806571006775 0.25629615783691406
-------------------------------
Round  1
Feedback:  The text is politically biased because it presents one side of the argument by suggesting that the impact on middle-class Americans and Main Street businesses would be negligible. To make it more neutral, the question could be rephrased as follows:

"What are some alternative solutions for raising $350 billion that would have a minimal impact on both middle-class Americans and Main Street businesses?"
Post-score: 0.47511088848114014 0.24456137418746948 0.280327707529068
-------------------------------
-------------------------------
Round  2
Feedback:  Great job on rephrasing the question to make it more neutral and balanced! The revised version presents the question in a way that does not assume the impact and allows for a more open discussion of potential solutions. This helps in promoting a fair and unbiased dialogue on the topic at hand.
Post-score: 0.3163769841

In [55]:
import pandas as pd

In [56]:
pd.DataFrame({"round": rounds, "left_score": left_scores})

,round,left_score
0,1,0.475111
1,2,0.316377
2,3,0.426321
3,4,0.506263
4,5,0.318703
5,6,0.295388
